In [10]:
import cupy as cp

# 1) CUDA source with temp1, temp2, alpha as arrays
kernel_code = r'''
extern "C" __global__
void update_p(const float* p0,
              const float* p1,
              const float* temp1,
              const float* temp2,
              const float* alpha,
              float*       pout,
              const int    nx,
              const int    ny,
              const float  c2,
              const float  c3) {
    int ix = blockDim.x * blockIdx.x + threadIdx.x;
    int iy = blockDim.y * blockIdx.y + threadIdx.y;
    if (ix >= nx || iy >= ny) return;
    int idx = iy * nx + ix;

    // periodic indexing macro
    #define IDX(i,j) ((( (i + nx) % nx) ) + nx * (( (j + ny) % ny) ))

    // gather neighbors for p1
    float t1 = p1[IDX(ix+1, iy)]
             + p1[IDX(ix-1, iy)]
             + p1[IDX(ix, iy+1)]
             + p1[IDX(ix, iy-1)];
    float t2 = p1[IDX(ix+2, iy)]
             + p1[IDX(ix-2, iy)]
             + p1[IDX(ix, iy+2)]
             + p1[IDX(ix, iy-2)];

    // use per-point temp1[idx], temp2[idx], alpha[idx]
    pout[idx] = temp1[idx] * p1[idx]
              - temp2[idx] * p0[idx]
              + alpha[idx] * (c2 * t1 + c3 * t2);

    #undef IDX
}
'''

# 2) Compile
module = cp.RawModule(code=kernel_code)
update_p = module.get_function('update_p')

# 3) Prepare data (example sizes and random fields)
nx, ny = 512, 512
shape = (ny, nx)
p0     = cp.random.rand(*shape).astype(cp.float32)
p1     = cp.random.rand(*shape).astype(cp.float32)
temp1  = cp.random.rand(*shape).astype(cp.float32)
temp2  = cp.random.rand(*shape).astype(cp.float32)
alpha  = cp.random.rand(*shape).astype(cp.float32)
pout   = cp.empty_like(p1)

# 4) Launch parameters
tx, ty = 16, 16
bx = (nx + tx - 1) // tx
by = (ny + ty - 1) // ty

# coefficient scalars
c2, c3 = 0.25, 0.125

# 5) Launch
update_p(
    (bx, by), (tx, ty),
    (
        p0.ravel(), p1.ravel(),
        temp1.ravel(), temp2.ravel(), alpha.ravel(),
        pout.ravel(),
        nx, ny,
        c2, c3
    )
)


# `pout` now contains the updated field.

p_alt = (temp1 * p1 - temp2 * p0 +
             alpha * (
                 c2 * (cp.roll(p1, 1, axis=1) + cp.roll(p1, -1, axis=1) +
                       cp.roll(p1, 1, axis=0) + cp.roll(p1, -1, axis=0)) +
                 c3 * (cp.roll(p1, 2, axis=1) + cp.roll(p1, -2, axis=1) +
                       cp.roll(p1, 2, axis=0) + cp.roll(p1, -2, axis=0))
             ))

cp.std(pout-p_alt),cp.std(pout)

(array(0.2353035, dtype=float32), array(0.3116985, dtype=float32))

In [1]:
import cupy as cp

# 1) CUDA source with explicit boundary checks
kernel_code = r'''
extern "C" __global__
void update_p(const float* __restrict__ p0,
              const float* __restrict__ p1,
              const float* __restrict__ temp1,
              const float* __restrict__ temp2,
              const float* __restrict__ alpha,
              float*             __restrict__ pout,
              const int    nx,
              const int    ny,
              const float  c2,
              const float  c3) {
    int ix = blockDim.x * blockIdx.x + threadIdx.x;
    int iy = blockDim.y * blockIdx.y + threadIdx.y;
    if (ix >= nx || iy >= ny) return;
    int idx = iy * nx + ix;

    // Compute wrapped neighbor indices by hand
    int ix_p1 = ix + 1; if (ix_p1 == nx)  ix_p1 = 0;
    int ix_m1 = ix - 1; if (ix_m1 < 0)    ix_m1 = nx - 1;
    int ix_p2 = ix + 2; if (ix_p2 >= nx)  ix_p2 -= nx;
    int ix_m2 = ix - 2; if (ix_m2 < 0)     ix_m2 += nx;

    int iy_p1 = iy + 1; if (iy_p1 == ny)  iy_p1 = 0;
    int iy_m1 = iy - 1; if (iy_m1 < 0)    iy_m1 = ny - 1;
    int iy_p2 = iy + 2; if (iy_p2 >= ny)  iy_p2 -= ny;
    int iy_m2 = iy - 2; if (iy_m2 < 0)     iy_m2 += ny;

    // gather the 4 nearest neighbors

    float t1 = p1[iy  * nx + ix_p1]
             + p1[iy  * nx + ix_m1]
             + p1[iy_p1 * nx + ix  ]
             + p1[iy_m1 * nx + ix  ];
    // gather the 4 next-nearest neighbors 
    float t2 = p1[iy  * nx + ix_p2]
             + p1[iy  * nx + ix_m2]
             + p1[iy_p2 * nx + ix  ]
             + p1[iy_m2 * nx + ix  ];

    // final update
    pout[idx] = temp1[idx] * p1[idx]
              - temp2[idx] * p0[idx]
              + alpha[idx] * (c2 * t1 + c3 * t2);
}
'''

# 2) Compile and get the function
module   = cp.RawModule(code=kernel_code)
update_p = module.get_function('update_p')

# 3) Host‐side driver and test
nx, ny = 512, 512
shape  = (ny, nx)

p0    = cp.random.rand(*shape, dtype=cp.float32)
p1    = cp.random.rand(*shape, dtype=cp.float32)
temp1 = cp.random.rand(*shape, dtype=cp.float32)
temp2 = cp.random.rand(*shape, dtype=cp.float32)
alpha = cp.random.rand(*shape, dtype=cp.float32)
pout  = cp.empty_like(p1)

# Launch configuration
tx, ty = 16, 16
bx = (nx + tx - 1) // tx
by = (ny + ty - 1) // ty

c2, c3 = 0.25, 0.125

# 4) Run the custom kernel
update_p((bx, by), (tx, ty),
         (p0.ravel(), p1.ravel(),
          temp1.ravel(), temp2.ravel(), alpha.ravel(),
          pout.ravel(),
          nx, ny, c2, c3))

# 5) Reference via cp.roll
p_ref = (temp1 * p1 - temp2 * p0 +
         alpha * (
             c2 * (cp.roll(p1,  1, axis=1) + cp.roll(p1, -1, axis=1) +
                   cp.roll(p1,  1, axis=0) + cp.roll(p1, -1, axis=0)) +
             c3 * (cp.roll(p1,  2, axis=1) + cp.roll(p1, -2, axis=1) +
                   cp.roll(p1,  2, axis=0) + cp.roll(p1, -2, axis=0))
         ))

# 6) Check
diff = pout - p_ref
print("max abs error:", float(cp.max(cp.abs(diff))))
print("relative  err:", float(cp.max(cp.abs(diff)) / cp.max(cp.abs(p_ref))))

max abs error: 1.2875655889511108
relative  err: 0.6566815376281738


In [2]:
import cupy as cp
import numpy as np

# 1) A tiny 8×8 grid so we can print out diagnostics:
nx, ny = 8, 8
shape = (ny, nx)

# 2) Create random test fields (but seed for reproducibility)
cp.random.seed(0)
p0    = cp.random.rand(*shape).astype(cp.float32)
p1    = cp.random.rand(*shape).astype(cp.float32)
temp1 = cp.random.rand(*shape).astype(cp.float32)
temp2 = cp.random.rand(*shape).astype(cp.float32)
alpha = cp.random.rand(*shape).astype(cp.float32)
pout  = cp.empty_like(p1)

c2, c3 = 0.25, 0.125

# 3) Our explicit‐boundary RawKernel
kernel_code = r'''
extern "C" __global__
void update_p(const float* __restrict__ p0,
              const float* __restrict__ p1,
              const float* __restrict__ temp1,
              const float* __restrict__ temp2,
              const float* __restrict__ alpha,
              float*             __restrict__ pout,
              const int    nx,
              const int    ny,
              const float  c2,
              const float  c3) {
    int ix = blockDim.x * blockIdx.x + threadIdx.x;
    int iy = blockDim.y * blockIdx.y + threadIdx.y;
    if (ix >= nx || iy >= ny) return;
    int idx = iy * nx + ix;

    // manual wrap at ±1, ±2
    int ix_p1 = ix+1; if (ix_p1==nx)  ix_p1=0;
    int ix_m1 = ix-1; if (ix_m1<0)    ix_m1=nx-1;
    int ix_p2 = ix+2; if (ix_p2>=nx)  ix_p2-=nx;
    int ix_m2 = ix-2; if (ix_m2<0)     ix_m2+=nx;
    int iy_p1 = iy+1; if (iy_p1==ny)  iy_p1=0;
    int iy_m1 = iy-1; if (iy_m1<0)    iy_m1=ny-1;
    int iy_p2 = iy+2; if (iy_p2>=ny)  iy_p2-=ny;
    int iy_m2 = iy-2; if (iy_m2<0)     iy_m2+=ny;

    // collect neighbors
    float t1 = p1[iy  * nx + ix_p1]
             + p1[iy  * nx + ix_m1]
             + p1[iy_p1 * nx + ix  ]
             + p1[iy_m1 * nx + ix  ];
    float t2 = p1[iy  * nx + ix_p2]
             + p1[iy  * nx + ix_m2]
             + p1[iy_p2 * nx + ix  ]
             + p1[iy_m2 * nx + ix  ];

    pout[idx] = temp1[idx] * p1[idx]
              - temp2[idx] * p0[idx]
              + alpha[idx] * (c2 * t1 + c3 * t2);
}
'''
module = cp.RawModule(code=kernel_code)
update_p = module.get_function('update_p')

# 4) Launch with a single block since 8×8 is small
tx, ty = 8, 8
bx, by = 1, 1
update_p((bx,by), (tx,ty),
         (p0.ravel(), p1.ravel(),
          temp1.ravel(), temp2.ravel(), alpha.ravel(),
          pout.ravel(),
          np.int32(nx), np.int32(ny), np.float32(c2), np.float32(c3)))

# 5) Reference via cp.roll
p_ref = (temp1 * p1 - temp2 * p0 +
         alpha * (
           c2*(cp.roll(p1, 1, axis=1) + cp.roll(p1, -1, axis=1) +
               cp.roll(p1, 1, axis=0) + cp.roll(p1, -1, axis=0)) +
           c3*(cp.roll(p1, 2, axis=1) + cp.roll(p1, -2, axis=1) +
               cp.roll(p1, 2, axis=0) + cp.roll(p1, -2, axis=0))
         ))

# 6) Compare
diff = pout - p_ref
print("p_ref:\n", p_ref.get())
print("p_out:\n", pout.get())
print("diff:\n", diff.get())
print("max abs err:", float(cp.max(cp.abs(diff))))
print("rel max err:", float(cp.max(cp.abs(diff)) / cp.max(cp.abs(p_ref))))

p_ref:
 [[ 0.668756    0.41473612  0.29223108  0.16266224  0.39287007  0.36644256
   0.21908289  0.12969482]
 [ 0.3179067   1.5124692   0.18974867  0.6998244  -0.10839958  0.60210466
   0.3902648   0.39430225]
 [ 0.598814   -0.07891804  0.47173256  0.52228373  0.23300076  1.0907717
  -0.07098675  0.47310784]
 [ 0.93139607  0.7828876   0.27592802  0.2684189  -0.3792216  -0.15253371
   0.4422996   0.19562839]
 [ 1.4504806  -0.46607327  0.679183    0.13941318  0.4781826  -0.11732632
   0.4220032   0.36692452]
 [ 0.38163006  0.14829956 -0.11520937  0.48957074  1.0951135   0.6737299
   0.990717    0.52804697]
 [ 0.9839347   0.46859238  0.29077494  0.19797882  0.61679614  0.71657723
   0.37520874  0.8263427 ]
 [ 0.71875304  0.37370825  0.21106216  0.62649155  0.7085141  -0.03221005
  -0.21509707  0.79537016]]
p_out:
 [[ 0.66875595  0.4147361   0.29223105  0.16266224  0.3928701   0.36644256
   0.21908289  0.12969482]
 [ 0.3179067   1.512469    0.18974869  0.69982433 -0.10839956  0.6021046
   

In [1]:
import cupy as cp
import numpy as np

# Problem size
nx, ny = 311,311
shape = (ny, nx)

# Create your fields (example random data; replace with your real arrays)
p0    = cp.random.rand(*shape).astype(cp.float32)
p1    = cp.random.rand(*shape).astype(cp.float32)
temp1 = cp.random.rand(*shape).astype(cp.float32)
temp2 = cp.random.rand(*shape).astype(cp.float32)
alpha = cp.random.rand(*shape).astype(cp.float32)
pout  = cp.empty_like(p1)

# Stencil coefficients
c2, c3 = np.float32(0.25), np.float32(0.125)

# The CUDA kernel with explicit boundary checks
kernel_code = r'''
extern "C" __global__
void update_p(const float* __restrict__ p0,
              const float* __restrict__ p1,
              const float* __restrict__ temp1,
              const float* __restrict__ temp2,
              const float* __restrict__ alpha,
              float*             __restrict__ pout,
              const int    nx,
              const int    ny,
              const float  c2,
              const float  c3) {
    int ix = blockDim.x * blockIdx.x + threadIdx.x;
    int iy = blockDim.y * blockIdx.y + threadIdx.y;
    if (ix >= nx || iy >= ny) return;
    int idx = iy * nx + ix;

    // Manual wrap at ±1, ±2
    int ix_p1 = ix+1; if (ix_p1==nx)  ix_p1=0;
    int ix_m1 = ix-1; if (ix_m1<0)    ix_m1=nx-1;
    int ix_p2 = ix+2; if (ix_p2>=nx)  ix_p2-=nx;
    int ix_m2 = ix-2; if (ix_m2<0)     ix_m2+=nx;
    int iy_p1 = iy+1; if (iy_p1==ny)  iy_p1=0;
    int iy_m1 = iy-1; if (iy_m1<0)    iy_m1=ny-1;
    int iy_p2 = iy+2; if (iy_p2>=ny)  iy_p2-=ny;
    int iy_m2 = iy-2; if (iy_m2<0)     iy_m2+=ny;

    // Collect neighbors (±1)
    float t1 = p1[iy  * nx + ix_p1]
             + p1[iy  * nx + ix_m1]
             + p1[iy_p1 * nx + ix  ]
             + p1[iy_m1 * nx + ix  ];
    // Collect neighbors (±2)
    float t2 = p1[iy  * nx + ix_p2]
             + p1[iy  * nx + ix_m2]
             + p1[iy_p2 * nx + ix  ]
             + p1[iy_m2 * nx + ix  ];

    pout[idx] = temp1[idx] * p1[idx]
              - temp2[idx] * p0[idx]
              + alpha[idx] * (c2 * t1 + c3 * t2);
}
'''

# Compile
module   = cp.RawModule(code=kernel_code)
update_p = module.get_function('update_p')

# Launch configuration (tunable: 16×16, 32×8, etc.)
tx, ty = 16, 16
bx = (nx + tx - 1) // tx
by = (ny + ty - 1) // ty

# Run the kernel
update_p(
    (bx, by), (tx, ty),
    (
        p0.ravel(), p1.ravel(),
        temp1.ravel(), temp2.ravel(), alpha.ravel(),
        pout.ravel(),
        np.int32(nx), np.int32(ny),
        c2, c3
    ),
)

# Optional check against cp.roll
p_ref = (temp1 * p1 - temp2 * p0 +
         alpha * (
             c2 * (cp.roll(p1,  1, axis=1) + cp.roll(p1, -1, axis=1) +
                   cp.roll(p1,  1, axis=0) + cp.roll(p1, -1, axis=0)) +
             c3 * (cp.roll(p1,  2, axis=1) + cp.roll(p1, -2, axis=1) +
                   cp.roll(p1,  2, axis=0) + cp.roll(p1, -2, axis=0))
         ))

diff = pout - p_ref
print("max abs error:", float(cp.max(cp.abs(diff))))
print("rel  error:",   float(cp.max(cp.abs(diff)) / cp.max(cp.abs(p_ref))))

max abs error: 2.384185791015625e-07
rel  error: 1.2707752716778487e-07
